# Step 1: Check Question Relevance

This notebook tests the first node in our RAG pipeline. Its purpose is to make an initial LLM call to determine if the user's question is related to emissions. This prevents the system from performing an unnecessary document search for off-topic questions.

In [1]:
import os
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# --- Configuration ---
MODEL_NAME = "llama3"
llm = Ollama(model=MODEL_NAME, temperature=0)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_62643/916712105.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=MODEL_NAME, temperature=0)


In [3]:
def check_relevance(state: dict, llm) -> dict:
    """
    Checks if the question is relevant to the document context.
    This function is defined locally for experimentation.
    """
    print("---CHECKING QUESTION RELEVANCE---")
    question = state['question']
    
    prompt_template = """
    You are an expert in oil and gas regulations. Your task is to determine if a user's question is related to 'oil and gas emissions' or 'emissions' in general.
    Answer with a simple 'yes' or 'no'.

    User question: "{question}"

    Is this question related to emissions? (yes/no):
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["question"])
    relevance_checker = prompt | llm | StrOutputParser()
    
    relevance_response = relevance_checker.invoke({"question": question})
    
    if "yes" in relevance_response.lower():
        print("---QUESTION IS RELEVANT---")
        return {"is_relevant": True}
    else:
        print("---QUESTION IS NOT RELEVANT---")
        return {"is_relevant": False}

### Test with a Relevant Question

In [4]:
relevant_question = "What are the monitoring requirements for flare gas?"
state = {"question": relevant_question}

result = check_relevance(state, llm)
print(result)

---CHECKING QUESTION RELEVANCE---
---QUESTION IS RELEVANT---
{'is_relevant': True}


### Test with an Irrelevant Question

In [5]:
irrelevant_question = "What is the capital of France?"
state = {"question": irrelevant_question}

result = check_relevance(state, llm)
print(result)

---CHECKING QUESTION RELEVANCE---
---QUESTION IS NOT RELEVANT---
{'is_relevant': False}
